In [1]:
# requirement packages
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install parsivar
!pip install num2fawords

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-g2eoag_c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-g2eoag_c
  Resolved https://github.com/huggingface/datasets.git to commit 31ae21ff806c3d1fc19a48ce41178c82d2f69368
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
  Created wheel for datasets: filename=datasets-2.17.2.dev0-py3-none-any.whl size=510578 sha256=1d7a1bcf1bed861b736458c94da1a89b2115772663d948c2d9c6906d40b3d67f
  Stored in directory: /tmp/pip-ephem-wheel-cache-56nvv3z9/wheels/57/f4/c4/53c677af89fec0ef3226c1e75a38367b37c2fa626f0544d3e4
Successfully built datasets
  Cloning https://github.com/huggingface/transformers.git to /tmp/pi

In [9]:
# Normalizer
!wget -O dictionary.py https://huggingface.co/m3hrdadfi/wav2vec2-large-xlsr-persian-v3/raw/main/dictionary.py
!wget -O normalizer.py https://huggingface.co/m3hrdadfi/wav2vec2-large-xlsr-persian-v3/raw/main/normalizer.py

--2024-03-01 20:39:15--  https://huggingface.co/m3hrdadfi/wav2vec2-large-xlsr-persian-v3/raw/main/dictionary.py
Resolving huggingface.co (huggingface.co)... 3.160.150.2, 3.160.150.119, 3.160.150.50, ...
Connecting to huggingface.co (huggingface.co)|3.160.150.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31456 (31K) [text/plain]
Saving to: ‘dictionary.py’

dictionary.py       100%[===================>]  30.72K  --.-KB/s    in 0.001s  

2024-03-01 20:39:15 (25.8 MB/s) - ‘dictionary.py’ saved [31456/31456]

--2024-03-01 20:39:15--  https://huggingface.co/m3hrdadfi/wav2vec2-large-xlsr-persian-v3/raw/main/normalizer.py
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.80, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6326 (6.2K) [text/plain]
Saving to: ‘normalizer.py’

normalizer.py       100%[===================>]   6.18K  --.-KB/s    in 

In [1]:
! ls

dictionary.py  normalizer.py  __pycache__  sample_data


In [14]:
! sed -i '1 s/^/from dictionary import dictionary_mapping, fixator_dictionary\n/' normalizer.py

In [15]:
! cat normalizer.py

from dictionary import dictionary_mapping
from parsivar import Normalizer

import num2fawords
import re
import string


_normalizer = Normalizer(half_space_char="\u200c", statistical_space_correction=True)
chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬", 'ٔ', ",", "?",
    ".", "!", "-", ";", ":", '"', "“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š', 'ّ', 'ْ',
]
chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)
chars_to_ignore = f"""[{"".join(chars_to_ignore)}]"""
zwnj = "\u200c"
silent_chars = ["ا", "د", "ذ", "ر", "ز", "و", "آ"] + [zwnj] + [" "]


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))


def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '',

In [2]:
from normalizer import normalizer

In [3]:
def cleaning(text):
    if not isinstance(text, str):
        return None

    return normalizer({"sentence": text}, return_dict=False)

In [5]:
cleaning("سلام حالتون خوبه؟")

'سلام حالتون خوبه '

In [6]:
import numpy as np
import pandas as pd

import librosa
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset, load_metric

import IPython.display as ipd

In [7]:
model_name_or_path = "m3hrdadfi/wav2vec2-large-xlsr-persian-v3"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model_name_or_path, device)

m3hrdadfi/wav2vec2-large-xlsr-persian-v3 cpu


In [8]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/307 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at m3hrdadfi/wav2vec2-large-xlsr-persian-v3 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing 

In [17]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=processor.feature_extractor.sampling_rate)

    return speech_array

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
speech_array = speech_file_to_array_fn("/content/drive/MyDrive/DML-AI-Audio-Speech/Resources/TestSamples/TestASREasy.wav")

In [18]:
def predict(speech):
    features = processor(
        speech,
        sampling_rate=processor.feature_extractor.sampling_rate,
        return_tensors="pt",
        padding=True
    )

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    pred_ids = torch.argmax(logits, dim=-1)

    predicted = processor.batch_decode(pred_ids)
    return predicted

In [20]:
predict(speech_array)

['این سوت برای آزمایش مدل تبدیل گفتار به مطن است']

In [21]:
speech_array = speech_file_to_array_fn("/content/drive/MyDrive/DML-AI-Audio-Speech/Resources/TestSamples/TestASRNormal.wav")
predict(speech_array)

['این سوت برای آزمایش مدل تبدیل گفتار به متن است']

In [22]:
speech_array = speech_file_to_array_fn("/content/drive/MyDrive/DML-AI-Audio-Speech/Resources/TestSamples/TestASRHard.wav")
predict(speech_array)

['این صبط برای آزمایش مدل تبدیل گفتار به مزنه']

In [23]:
speech_array = speech_file_to_array_fn("/content/drive/MyDrive/DML-AI-Audio-Speech/Resources/TestSamples/TestASRBeGhasdeKosht.wav")
predict(speech_array)

['واقعا انتظار نداشتن مدل انقدر خوب عمل کنه']